In [1]:
import sys
import importlib

# Remove the module from sys.modules
if 'varseek.build' in sys.modules:
    del sys.modules['varseek.build']

# Import the module and reload it
import varseek.varseek_build
importlib.reload(varseek.varseek_build)
import varseek as vk

# instead of using vk.build() as before, I must explicitely use vk.varseek_build.build()

In [ ]:
vk.varseek_build.build(
    variants="cosmic_cmc",
    sequences="cdna",
    out="trash/vk_build_test",
    gtf=True,
    required_insertion_overlap_length=None,  #!!! erase this line eventually
)